In [1]:
import pandas as pd
import time
import os
import pickle

import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from keras.callbacks import ModelCheckpoint

import numpy as np

## 데이터 전처리

In [21]:
good_data = pd.read_csv("./dataset/good.csv")
bad_data = pd.read_csv("./dataset/bad.csv")

good_data = good_data.astype(int)
bad_data = bad_data.astype(int)

if os.path.exists(f'./dataset/fix_good_datas') and os.path.exists(f'./dataset/fix_bad_data') and os.path.exists(f'./dataset/drop_cols'):
    fix_good_data = pickle.load(open(f'./fix_good_data','rb'))
    fix_bad_data = pickle.load(open(f'./fix_bad_data','rb'))
    drop_cols = pickle.load(open(f'./drop_cols','rb'))
    
else:

    drop_cols = ['Unnamed: 0', '.', '+', '-',  '/', ',', '및', '은', '을', '의', '첫', '하기', '를', '3', '된', '매', 
                 '10', '5', '에', '에서', '완료', '!', '과', '있습니다', '만', '1', '8', '50', '는', '가', '고', '가장',
                  '=', '2', '중','한', '하고', '4', '위', ')', '(', '100','30', '?', '하여', 'ㄴ', '것', '입니다', '많은',
                  '합니다', '6', '시', '모든', '다', '와', '곳', '하는', '도', '적', '인', '할', '즈', '_', '경우',
                  '우리', '일', '더', '9', '7', '0', '수', '들', ':', ']', '[', '으로', '없는', '&', '등', '로', '있는',
                   '이', '충', '|', '\'', '\'', '\"', '경기', '저희', '호텔', '365']
    for col in bad_data.columns:

        vacuum_check = col.strip()

        if bad_data[col].sum() < 5000 or len(vacuum_check) == 0:
            drop_cols.append(col)

    fix_good_data = good_data.drop(['Unnamed: 0.1'], axis=1)
    fix_good_data = fix_good_data.drop(drop_cols, 1)
    fix_bad_data = bad_data.drop(drop_cols, 1)

    fix_good_data['label'] = 0
    fix_bad_data['label'] = 1

    pickle.dump(fix_good_data, open(f'./dataset/fix_good_data','wb'))
    pickle.dump(fix_bad_data, open(f'./dataset/fix_bad_data','wb'))
    pickle.dump(drop_cols, open(f'./dataset/drop_cols','wb'))

## 데이터셋 구축

In [22]:
# 데이터셋 나누기 학습용 90% 테스트용 10%
good_train = fix_good_data.sample(frac=0.9)
good_test = fix_good_data.drop(good_train.index)

bad_train = fix_bad_data.sample(frac=0.9)
bad_test = fix_bad_data.drop(bad_train.index)

train = pd.concat([good_train, bad_train], axis=0)
test = pd.concat([good_test, bad_test], axis=0)

# Shuffle
train = train.sample(frac=1)
test = test.sample(frac=1)

x_train = train.iloc[:, :-1].to_numpy()
y_train = train.iloc[:, -1:].to_numpy()

x_test = test.iloc[:, :-1].to_numpy()
y_test = test.iloc[:, -1:].to_numpy()

In [35]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

checkpoint_path = 'dataset/cp'

checkpoint = ModelCheckpoint(checkpoint_path, 
                            save_best_only=True, 
                            save_weights_only=True, 
                            monitor='val_loss',
                            verbose=1)


model = Sequential([

Dense(128, activation='relu' ,input_shape=[14]),
Dense(64, activation='relu'),
Dense(32, activation='relu'),

    Dense(2, activation='softmax')
])

model.compile(optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

model.fit(x_train, y_train, validation_data=(x_test, y_test) , shuffle=True, epochs=100, verbose=1, callbacks=[checkpoint],)

Epoch 1/100
28/28 [==============================] - 0s 8ms/step - loss: 1.5107 - accuracy: 0.6922 - val_loss: 0.4259 - val_accuracy: 0.9082

Epoch 00001: val_loss improved from inf to 0.42593, saving model to dataset/cp
Epoch 2/100
28/28 [==============================] - 0s 4ms/step - loss: 0.3103 - accuracy: 0.9211 - val_loss: 0.2731 - val_accuracy: 0.9388

Epoch 00002: val_loss improved from 0.42593 to 0.27314, saving model to dataset/cp
Epoch 3/100
28/28 [==============================] - 0s 5ms/step - loss: 0.2225 - accuracy: 0.9382 - val_loss: 0.2738 - val_accuracy: 0.9388

Epoch 00003: val_loss did not improve from 0.27314
Epoch 4/100
28/28 [==============================] - 0s 5ms/step - loss: 0.1926 - accuracy: 0.9405 - val_loss: 0.2279 - val_accuracy: 0.9388

Epoch 00004: val_loss improved from 0.27314 to 0.22787, saving model to dataset/cp
Epoch 5/100
28/28 [==============================] - 0s 5ms/step - loss: 0.1682 - accuracy: 0.9451 - val_loss: 0.2343 - val_accuracy: 0.

28/28 [==============================] - 0s 5ms/step - loss: 0.0978 - accuracy: 0.9634 - val_loss: 0.2934 - val_accuracy: 0.9388

Epoch 00042: val_loss did not improve from 0.15113
Epoch 43/100
28/28 [==============================] - 0s 5ms/step - loss: 0.0976 - accuracy: 0.9645 - val_loss: 0.2355 - val_accuracy: 0.9490

Epoch 00043: val_loss did not improve from 0.15113
Epoch 44/100
28/28 [==============================] - 0s 5ms/step - loss: 0.0957 - accuracy: 0.9645 - val_loss: 0.2343 - val_accuracy: 0.9490

Epoch 00044: val_loss did not improve from 0.15113
Epoch 45/100
28/28 [==============================] - 0s 5ms/step - loss: 0.0943 - accuracy: 0.9645 - val_loss: 0.3170 - val_accuracy: 0.9388

Epoch 00045: val_loss did not improve from 0.15113
Epoch 46/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0937 - accuracy: 0.9645 - val_loss: 0.2309 - val_accuracy: 0.9490

Epoch 00046: val_loss did not improve from 0.15113
Epoch 47/100
28/28 [=======================

28/28 [==============================] - 0s 5ms/step - loss: 0.0881 - accuracy: 0.9680 - val_loss: 0.2799 - val_accuracy: 0.9490

Epoch 00084: val_loss did not improve from 0.14458
Epoch 85/100
28/28 [==============================] - 0s 5ms/step - loss: 0.0873 - accuracy: 0.9668 - val_loss: 0.2756 - val_accuracy: 0.9490

Epoch 00085: val_loss did not improve from 0.14458
Epoch 86/100
28/28 [==============================] - 0s 5ms/step - loss: 0.0902 - accuracy: 0.9668 - val_loss: 0.2560 - val_accuracy: 0.9490

Epoch 00086: val_loss did not improve from 0.14458
Epoch 87/100
28/28 [==============================] - 0s 5ms/step - loss: 0.0865 - accuracy: 0.9657 - val_loss: 0.2926 - val_accuracy: 0.9490

Epoch 00087: val_loss did not improve from 0.14458
Epoch 88/100
28/28 [==============================] - 0s 5ms/step - loss: 0.0862 - accuracy: 0.9668 - val_loss: 0.2867 - val_accuracy: 0.9490

Epoch 00088: val_loss did not improve from 0.14458
Epoch 89/100
28/28 [=======================

In [36]:
print(model.evaluate(x_test, y_test)[1])

4/4 [==============================] - 0s 2ms/step - loss: 0.2965 - accuracy: 0.9490
0.9489796161651611


In [ ]:
good_train.sum()

In [ ]:
bad_train.sum()